In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tqdm.notebook import tqdm
from skimage.registration import optical_flow_tvl1, optical_flow_ilk
from skimage.transform import warp
from skimage.color import rgb2gray
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import normalized_root_mse as nrmse

In [2]:
def retrieve_images(patient_id = '156518', laterality = 'L'):
    # Set the root directory for the patient data
    root_dir = Path(f'../data/{patient_id}')

    # Get the list of image filenames for the left eye
    image_filenames = [f for f in os.listdir(root_dir) if f'{laterality}.png' in f]

    # Read the images into a list
    images = [cv2.imread(str(root_dir / f)) for f in image_filenames]

    # Convert the images to grayscale
    gray_images = [rgb2gray(img) for img in images]

    # Register all images to the first image
    template = gray_images[0]

    # Remove invalid images
    final_images = [x for x in gray_images[1:] if x.shape == template.shape]

    return final_images, template

In [6]:
import SimpleITK as sitk
import numpy as np
import os
OUTPUT_DIR = 'output'

In [7]:
images, template = retrieve_images()

In [9]:
fixed_image =  sitk.GetImageFromArray(template, sitk.sitkFloat32)
moving_image = sitk.GetImageFromArray(images[0], sitk.sitkFloat32)

In [10]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

RuntimeError: Exception thrown in SimpleITK CenteredTransformInitializer: /tmp/SimpleITK/Code/BasicFilters/src/sitkCenteredTransformInitializerFilter.cxx:87:
sitk::ERROR: Transform parameter for CenteredTransformInitializerFilter doesn't match dimension!